In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [38]:
query = (
    '''
    SELECT
  *,
  YEAR(contract_date)-TOP AS lasting_year
FROM (
  SELECT
    UPPER(property_name) AS upper_name,
    TOP,
    district,
    units,
    property_type,
    LOWER(tenure) AS lower_tenure,
    FROM
    [drq-machine-learning:enbloc.property_list] )AS property_list
LEFT JOIN (
  SELECT
    contract_date,
    project_name,
    1 AS is_enbloc
  FROM
    [drq-machine-learning:enbloc.enbloc]) AS enbloc_list
ON
  property_list.upper_name = enbloc_list.project_name
  
WHERE 
property_list.property_type!='Office'
    '''
)
df = pd.read_gbq(
    query,
    project_id='drq-machine-learning',
    private_key='/Users/zhouwanjie/Documents/Projects/enbloc/drq-machine-learning-0a5d30a93870.json',
    dialect='legacy'
)

Requesting query... ok.
Job ID: 3051d0ff-2e3f-483c-a042-ffc98305f5a1
Query running...
Query done.
Cache hit.

Retrieving results...
Got 2137 rows.

Total time taken 3.5 s.
Finished at 2018-04-15 11:12:37.


In [39]:
df.head()

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year
0,STARLIGHT ROAD,None,D8 - Little India,None,Apartment,9999 years from 01/01/1957,None,None,None,None
1,CAMBRIDGE ROAD,None,D8 - Little India,None,Apartment,999999 years,None,None,None,None
2,JALAN SENANG,None,"D14 - Geylang, Eunos",None,Apartment,999 years from 01/01/1967,None,None,None,None
3,GEK LIM MANSIONS,None,"D14 - Geylang, Eunos",None,Apartment,n.a,None,None,None,None
4,LORONG 24 GEYLANG,None,"D14 - Geylang, Eunos",None,Apartment,9999 years from 01/01/1962,None,None,None,None


In [40]:
df['property_list_lower_tenure'].unique()

array([u'9999 years from 01/01/1957', u'999999 years',
       u'999 years from 01/01/1967', u'n.a',
       u'9999 years from 01/01/1962', u'110 years from 01/11/2017',
       u'999 years', u'99 years from 01/03/1977',
       u'99 years from 25/03/1968', u'99 years from 04/08/1969',
       u'99 years from 02/06/1970', u'99 years from 18/01/1972',
       u'99 years from 03/09/1968', u'99 years from 01/01/1969',
       u'99 years from 30/08/1976', u'99 years from 01/11/1975',
       u'999 years from 07/06/1884', u'99 years from 18/09/1979',
       u'99 years from 16/06/1980', u'99 years from 15/12/1980',
       u'999 years from 25/01/1827', None, u'9999 years from 02/06/1995',
       u'999 years from 29/05/1885', u'99 years from 08/09/1992',
       u'99 years from 01/01/1995', u'999 years from 02/06/1882',
       u'99 years from 17/11/1993', u'99 years from 02/10/1992',
       u'leasehold/99 years', u'929 years from 01/01/1953',
       u'99 years from 13/02/1995', u'99 years from 06/11/19

In [41]:
df['property_list_lower_tenure'] = df['property_list_lower_tenure'].apply(lambda x: str(x))

In [42]:
df = df[df['property_list_lower_tenure'].notnull()]

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2137 entries, 0 to 2136
Data columns (total 10 columns):
property_list_upper_name       2137 non-null object
property_list_TOP              2041 non-null object
property_list_district         2137 non-null object
property_list_units            1928 non-null object
property_list_property_type    2137 non-null object
property_list_lower_tenure     2137 non-null object
enbloc_list_contract_date      319 non-null object
enbloc_list_project_name       319 non-null object
enbloc_list_is_enbloc          319 non-null object
lasting_year                   281 non-null object
dtypes: object(10)
memory usage: 183.6+ KB


In [44]:
df=df[df['property_list_lower_tenure']!='n.a']

In [45]:
df.head()

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year
0,STARLIGHT ROAD,None,D8 - Little India,None,Apartment,9999 years from 01/01/1957,None,None,None,None
1,CAMBRIDGE ROAD,None,D8 - Little India,None,Apartment,999999 years,None,None,None,None
2,JALAN SENANG,None,"D14 - Geylang, Eunos",None,Apartment,999 years from 01/01/1967,None,None,None,None
4,LORONG 24 GEYLANG,None,"D14 - Geylang, Eunos",None,Apartment,9999 years from 01/01/1962,None,None,None,None
8,PARKSUITES,1900,"D10 - Bukit Timah, Holland Rd, Tanglin",None,Apartment,110 years from 01/11/2017,None,None,None,None


In [46]:
df['use_year'] = df['property_list_lower_tenure'].apply(lambda x: x.split(' ')[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2128 non-null object
property_list_TOP              2037 non-null object
property_list_district         2128 non-null object
property_list_units            1928 non-null object
property_list_property_type    2128 non-null object
property_list_lower_tenure     2128 non-null object
enbloc_list_contract_date      319 non-null object
enbloc_list_project_name       319 non-null object
enbloc_list_is_enbloc          319 non-null object
lasting_year                   281 non-null object
use_year                       2128 non-null object
dtypes: object(11)
memory usage: 199.5+ KB


In [47]:
df['use_year'].unique()

array(['9999', '999999', '999', '110', '99', 'None', 'leasehold/99',
       '929', '947', '946', '956', '60', 'freehold', '101', '100', '103',
       '99-year', 'f', 'l99', 'l999', 'mixed'], dtype=object)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2128 non-null object
property_list_TOP              2037 non-null object
property_list_district         2128 non-null object
property_list_units            1928 non-null object
property_list_property_type    2128 non-null object
property_list_lower_tenure     2128 non-null object
enbloc_list_contract_date      319 non-null object
enbloc_list_project_name       319 non-null object
enbloc_list_is_enbloc          319 non-null object
lasting_year                   281 non-null object
use_year                       2128 non-null object
dtypes: object(11)
memory usage: 199.5+ KB


In [49]:
df['property_list_TOP']=df['property_list_TOP'].fillna(df['property_list_TOP'].median())

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2128 non-null object
property_list_TOP              2128 non-null float64
property_list_district         2128 non-null object
property_list_units            1928 non-null object
property_list_property_type    2128 non-null object
property_list_lower_tenure     2128 non-null object
enbloc_list_contract_date      319 non-null object
enbloc_list_project_name       319 non-null object
enbloc_list_is_enbloc          319 non-null object
lasting_year                   281 non-null object
use_year                       2128 non-null object
dtypes: float64(1), object(10)
memory usage: 199.5+ KB


In [56]:
df['lasting_year'][df['enbloc_list_is_enbloc'].notnull()]=df['lasting_year'][df['enbloc_list_is_enbloc'].notnull()].fillna(df['lasting_year'].median())

/Users/zhouwanjie/virtualenvironment/enbloc/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2128 non-null object
property_list_TOP              2128 non-null float64
property_list_district         2128 non-null object
property_list_units            1928 non-null object
property_list_property_type    2128 non-null object
property_list_lower_tenure     2128 non-null object
enbloc_list_contract_date      319 non-null object
enbloc_list_project_name       319 non-null object
enbloc_list_is_enbloc          319 non-null object
lasting_year                   319 non-null object
use_year                       2128 non-null object
dtypes: float64(1), object(10)
memory usage: 279.5+ KB


In [58]:
df

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year,use_year
0,STARLIGHT ROAD,2004.0,D8 - Little India,None,Apartment,9999 years from 01/01/1957,None,None,None,None,9999
1,CAMBRIDGE ROAD,2004.0,D8 - Little India,None,Apartment,999999 years,None,None,None,None,999999
2,JALAN SENANG,2004.0,"D14 - Geylang, Eunos",None,Apartment,999 years from 01/01/1967,None,None,None,None,999
4,LORONG 24 GEYLANG,2004.0,"D14 - Geylang, Eunos",None,Apartment,9999 years from 01/01/1962,None,None,None,None,9999
8,PARKSUITES,1900.0,"D10 - Bukit Timah, Holland Rd, Tanglin",None,Apartment,110 years from 01/11/2017,None,None,None,None,110
9,TEACHERS' HOUSING ESTATE,1968.0,"D26 - Upper Thomson, Springleaf",None,Apartment,999 years,None,None,None,None,999
10,LAKESIDE APARTMENTS,1970.0,D22 - Jurong,None,Apartment,99 years from 01/03/1977,None,None,None,None,99
11,PEOPLE'S PARK COMPLEX,1972.0,"D1 - Temasek Blvd, Raffles Link",None,Apartment,99 years from 25/03/1968,None,None,None,None,99
12,GOLDEN MILE COMPLEX,1974.0,"D7 - Middle Road, Golden Mile",None,Apartment,99 years from 04/08/1969,None,None,None,None,99
13,INTERNATIONAL PLAZA,1976.0,"D2 - Anson, Tanjong Pagar",None,Apartment,99 years from 02/06/1970,None,None,None,None,99


In [59]:
df['lasting_year'][df['enbloc_list_is_enbloc'].isnull()]=2018-df['property_list_TOP']

/Users/zhouwanjie/virtualenvironment/enbloc/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
df

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year,use_year
0,STARLIGHT ROAD,2004.0,D8 - Little India,None,Apartment,9999 years from 01/01/1957,None,None,None,14,9999
1,CAMBRIDGE ROAD,2004.0,D8 - Little India,None,Apartment,999999 years,None,None,None,14,999999
2,JALAN SENANG,2004.0,"D14 - Geylang, Eunos",None,Apartment,999 years from 01/01/1967,None,None,None,14,999
4,LORONG 24 GEYLANG,2004.0,"D14 - Geylang, Eunos",None,Apartment,9999 years from 01/01/1962,None,None,None,14,9999
8,PARKSUITES,1900.0,"D10 - Bukit Timah, Holland Rd, Tanglin",None,Apartment,110 years from 01/11/2017,None,None,None,118,110
9,TEACHERS' HOUSING ESTATE,1968.0,"D26 - Upper Thomson, Springleaf",None,Apartment,999 years,None,None,None,50,999
10,LAKESIDE APARTMENTS,1970.0,D22 - Jurong,None,Apartment,99 years from 01/03/1977,None,None,None,48,99
11,PEOPLE'S PARK COMPLEX,1972.0,"D1 - Temasek Blvd, Raffles Link",None,Apartment,99 years from 25/03/1968,None,None,None,46,99
12,GOLDEN MILE COMPLEX,1974.0,"D7 - Middle Road, Golden Mile",None,Apartment,99 years from 04/08/1969,None,None,None,44,99
13,INTERNATIONAL PLAZA,1976.0,"D2 - Anson, Tanjong Pagar",None,Apartment,99 years from 02/06/1970,None,None,None,42,99


In [62]:
df['property_list_district'] = df['property_list_district'].apply(lambda x: x.split(' - ')[0])

In [63]:
df

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year,use_year
0,STARLIGHT ROAD,2004.0,D8,None,Apartment,9999 years from 01/01/1957,None,None,None,14,9999
1,CAMBRIDGE ROAD,2004.0,D8,None,Apartment,999999 years,None,None,None,14,999999
2,JALAN SENANG,2004.0,D14,None,Apartment,999 years from 01/01/1967,None,None,None,14,999
4,LORONG 24 GEYLANG,2004.0,D14,None,Apartment,9999 years from 01/01/1962,None,None,None,14,9999
8,PARKSUITES,1900.0,D10,None,Apartment,110 years from 01/11/2017,None,None,None,118,110
9,TEACHERS' HOUSING ESTATE,1968.0,D26,None,Apartment,999 years,None,None,None,50,999
10,LAKESIDE APARTMENTS,1970.0,D22,None,Apartment,99 years from 01/03/1977,None,None,None,48,99
11,PEOPLE'S PARK COMPLEX,1972.0,D1,None,Apartment,99 years from 25/03/1968,None,None,None,46,99
12,GOLDEN MILE COMPLEX,1974.0,D7,None,Apartment,99 years from 04/08/1969,None,None,None,44,99
13,INTERNATIONAL PLAZA,1976.0,D2,None,Apartment,99 years from 02/06/1970,None,None,None,42,99


In [67]:
df['property_list_units']=df['property_list_units'].fillna(df['property_list_units'].median())
df['enbloc_list_is_enbloc']=df['enbloc_list_is_enbloc'].fillna(0)

In [68]:
df

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year,use_year
0,STARLIGHT ROAD,2004.0,D8,70.0,Apartment,9999 years from 01/01/1957,None,None,0,14,9999
1,CAMBRIDGE ROAD,2004.0,D8,70.0,Apartment,999999 years,None,None,0,14,999999
2,JALAN SENANG,2004.0,D14,70.0,Apartment,999 years from 01/01/1967,None,None,0,14,999
4,LORONG 24 GEYLANG,2004.0,D14,70.0,Apartment,9999 years from 01/01/1962,None,None,0,14,9999
8,PARKSUITES,1900.0,D10,70.0,Apartment,110 years from 01/11/2017,None,None,0,118,110
9,TEACHERS' HOUSING ESTATE,1968.0,D26,70.0,Apartment,999 years,None,None,0,50,999
10,LAKESIDE APARTMENTS,1970.0,D22,70.0,Apartment,99 years from 01/03/1977,None,None,0,48,99
11,PEOPLE'S PARK COMPLEX,1972.0,D1,70.0,Apartment,99 years from 25/03/1968,None,None,0,46,99
12,GOLDEN MILE COMPLEX,1974.0,D7,70.0,Apartment,99 years from 04/08/1969,None,None,0,44,99
13,INTERNATIONAL PLAZA,1976.0,D2,70.0,Apartment,99 years from 02/06/1970,None,None,0,42,99


In [69]:
data = df[['property_list_upper_name', 'property_list_district', 'property_list_units',
           'property_list_property_type', 'enbloc_list_is_enbloc', 'lasting_year', 'use_year']]

In [71]:
data = pd.get_dummies(data, columns=['property_list_district', 'property_list_property_type', 'use_year'])

In [72]:
data

,property_list_upper_name,property_list_units,enbloc_list_is_enbloc,lasting_year,property_list_district_D02,property_list_district_D03,property_list_district_D04,property_list_district_D05,property_list_district_D07,property_list_district_D08,...,use_year_999,use_year_9999,use_year_999999,use_year_None,use_year_f,use_year_freehold,use_year_l99,use_year_l999,use_year_leasehold/99,use_year_mixed
0,STARLIGHT ROAD,70.0,0,14,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,CAMBRIDGE ROAD,70.0,0,14,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,JALAN SENANG,70.0,0,14,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,LORONG 24 GEYLANG,70.0,0,14,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,PARKSUITES,70.0,0,118,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,TEACHERS' HOUSING ESTATE,70.0,0,50,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10,LAKESIDE APARTMENTS,70.0,0,48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,PEOPLE'S PARK COMPLEX,70.0,0,46,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,GOLDEN MILE COMPLEX,70.0,0,44,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,INTERNATIONAL PLAZA,70.0,0,42,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
x = data.drop(columns=['property_list_upper_name', 'enbloc_list_is_enbloc']).as_matrix()
y = data['enbloc_list_is_enbloc'].as_matrix()

In [84]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2)

In [85]:
x_train.shape

(1702, 66)

In [86]:
x_test.shape

(426, 66)

In [90]:
x.shape


(2128, 66)

In [92]:
random_forest=RandomForestClassifier(random_state=10)
random_forest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [102]:
y_predict=random_forest.predict(x_test)

In [103]:
acc =metrics.accuracy_score(y_test,y_predict)

In [104]:
print(acc)

0.9882629107981221


In [105]:
pre =metrics.precision_score(y_test,y_predict)
print (pre)

1.0


In [106]:
metrics.recall_score(y_test,y_predict)

0.9180327868852459